In [236]:
%run ./CFLP_Cplex.ipynb

In [237]:
# 需要点と施設候補の座標データ
# demand_points = [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5)]
# candidate_sites = [(10, 10), (20, 20), (30, 30), (40, 40), (50, 50)]

demand_points = [
    (2, 22), (42, 6), (48, 50), (32, 40), (16, 10),
]
candidate_sites = [
    (10, 13), (47, 16), (30, 44), (28, 47), (6, 1),
]


J = len(candidate_sites)  # 施設候補の数
D = len(demand_points)  # 需要点の数
alpha = 0
beta = 0.1

p = 2
r = 2

h_i = np.full(D, 1 / D)

# 既存のリーダーの施設セット J_L を仮定
J_L = {}  # インデックスとして候補施設の一部を選択
J_F = {}


In [238]:
distances = compute_distances(demand_points, candidate_sites)

# w_ij の計算
wij_matrix = compute_wij_matrix(distances, alpha, beta)


# U_i^L の計算
Ui_L = compute_Ui_L(wij_matrix, J_L)

# U_i^F の計算
Ui_F = compute_Ui_F(wij_matrix, J_F)

In [239]:
# グローバル変数を定義
F = []  # 問題のセット
BestSol = None  # 最良解
theta_LB = 0  # 下限値

In [240]:
# 追加されたカット制約を記録するセット
existing_cuts = set()

# compute_distancesの検証

In [241]:
# ====== テストの実行 ======
print("🔍 [Test 1] 基本的なケース")
demand_points = [(0, 0), (1, 1)]
candidate_sites = [(1, 0), (2, 2)]
expected = np.array([
    [1.0, np.sqrt(8)],  # (0,0) → (1,0) = 1, (0,0) → (2,2) = sqrt(8)
    [1.0, np.sqrt(2)]    # (1,1) → (1,0) = 1, (1,1) → (2,2) = sqrt(2)
])
result = compute_distances(demand_points, candidate_sites)

if np.allclose(result, expected):
    print("✅ Test 1 Passed!")
else:
    print(f"❌ Test 1 Failed! \nExpected:\n{expected}\nGot:\n{result}")

print("\n🔍 [Test 2] ゼロ距離チェック")
demand_points = [(2, 2)]
candidate_sites = [(2, 2)]
expected = np.array([[0.0]])  # 同じ座標なので距離は0
result = compute_distances(demand_points, candidate_sites)

if np.allclose(result, expected):
    print("✅ Test 2 Passed!")
else:
    print(f"❌ Test 2 Failed! \nExpected:\n{expected}\nGot:\n{result}")

print("\n🔍 [Test 3] 対称性チェック")
demand_points = [(1, 0)]
candidate_sites = [(0, 0), (2, 0)]
expected = np.array([[1.0, 1.0]])  # (1,0) → (0,0) = 1, (1,0) → (2,0) = 1
result = compute_distances(demand_points, candidate_sites)

if np.allclose(result, expected):
    print("✅ Test 3 Passed!")
else:
    print(f"❌ Test 3 Failed! \nExpected:\n{expected}\nGot:\n{result}")

print("\n🔍 [Test 4] 大きな値のチェック")
demand_points = [(1000, 2000)]
candidate_sites = [(3000, 4000)]
expected = np.array([[np.sqrt(8000000)]])  # sqrt((3000-1000)^2 + (4000-2000)^2)
result = compute_distances(demand_points, candidate_sites)

if np.allclose(result, expected):
    print("✅ Test 4 Passed!")
else:
    print(f"❌ Test 4 Failed! \nExpected:\n{expected}\nGot:\n{result}")

🔍 [Test 1] 基本的なケース
✅ Test 1 Passed!

🔍 [Test 2] ゼロ距離チェック
✅ Test 2 Passed!

🔍 [Test 3] 対称性チェック
✅ Test 3 Passed!

🔍 [Test 4] 大きな値のチェック
✅ Test 4 Passed!


# compute_wij_matrixのテスト

In [242]:
# ====== テスト 1：基本的な距離からの w_ij 計算 ======
print("🔍 [Test 1] 基本的なケース")
distances = np.array([
    [0.0, 10.0],
    [5.0, 0.0]
])
alpha = 0
beta = 0.1
expected = np.array([
    [np.exp(0), np.exp(-1.0)],  # exp(0), exp(-0.1 * 10)
    [np.exp(-0.5), np.exp(0)]   # exp(-0.1 * 5), exp(0)
])
result = compute_wij_matrix(distances, alpha, beta)

if np.allclose(result, expected):
    print("✅ Test 1 Passed!")
else:
    print(f"❌ Test 1 Failed!\nExpected:\n{expected}\nGot:\n{result}")

# ====== テスト 2：α, β を変化させたケース ======
print("\n🔍 [Test 2] alpha = 1, beta = 0.2 のケース")
alpha = 1
beta = 0.2
expected = np.exp(alpha - beta * distances)
result = compute_wij_matrix(distances, alpha, beta)

if np.allclose(result, expected):
    print("✅ Test 2 Passed!")
else:
    print(f"❌ Test 2 Failed!\nExpected:\n{expected}\nGot:\n{result}")

# ====== テスト 3：ゼロ距離での最大値チェック ======
print("\n🔍 [Test 3] 距離ゼロのチェック")
distances = np.array([[0.0]])
expected = np.array([[np.exp(alpha)]])  # alpha=1, beta=0.2 のまま
result = compute_wij_matrix(distances, alpha, beta)

if np.allclose(result, expected):
    print("✅ Test 3 Passed!")
else:
    print(f"❌ Test 3 Failed!\nExpected:\n{expected}\nGot:\n{result}")


🔍 [Test 1] 基本的なケース
✅ Test 1 Passed!

🔍 [Test 2] alpha = 1, beta = 0.2 のケース
✅ Test 2 Passed!

🔍 [Test 3] 距離ゼロのチェック
✅ Test 3 Passed!


In [243]:
import numpy as np

def compute_Ui_L(wij_matrix, J_L):
    D, J = wij_matrix.shape
    if not J_L:
        return np.zeros(D)
    utility_vector = np.zeros(D)
    for j in J_L:
        utility_vector += wij_matrix[:, j]
    return utility_vector

# ====== Test 1: 単一施設 ======
print("🔍 [Test 1] 単一施設のチェック")
wij_matrix = np.array([
    [0.1, 0.2, 0.3],
    [0.4, 0.5, 0.6],
])
J_L = {1}  # 施設1番のみ
expected = np.array([0.2, 0.5])
result = compute_Ui_L(wij_matrix, J_L)

if np.allclose(result, expected):
    print("✅ Test 1 Passed!")
    print(result)
else:
    print(f"❌ Test 1 Failed!\nExpected: {expected}\nGot: {result}")

# ====== Test 2: 複数施設 ======
print("\n🔍 [Test 2] 複数施設の合計チェック")
J_L = {0, 2}  # 施設0と2
expected = np.array([0.1 + 0.3, 0.4 + 0.6])  # = [0.4, 1.0]
result = compute_Ui_L(wij_matrix, J_L)

if np.allclose(result, expected):
    print("✅ Test 2 Passed!")
    print(result)
else:
    print(f"❌ Test 2 Failed!\nExpected: {expected}\nGot: {result}")

# ====== Test 3: 空集合 ======
print("\n🔍 [Test 3] 施設なし（空集合）のチェック")
J_L = set()
expected = np.array([0.0, 0.0])
result = compute_Ui_L(wij_matrix, J_L)

if np.allclose(result, expected):
    print("✅ Test 3 Passed!")
    print(result)
else:
    print(f"❌ Test 3 Failed!\nExpected: {expected}\nGot: {result}")

# ====== Test 4: 全施設を選ぶ ======
print("\n🔍 [Test 4] 全施設を使ったときの合計チェック")
J_L = {0, 1, 2}
expected = wij_matrix.sum(axis=1)
result = compute_Ui_L(wij_matrix, J_L)

if np.allclose(result, expected):
    print("✅ Test 4 Passed!")
    print(result)
else:
    print(f"❌ Test 4 Failed!\nExpected: {expected}\nGot: {result}")


🔍 [Test 1] 単一施設のチェック
✅ Test 1 Passed!
[0.2 0.5]

🔍 [Test 2] 複数施設の合計チェック
✅ Test 2 Passed!
[0.4 1. ]

🔍 [Test 3] 施設なし（空集合）のチェック
✅ Test 3 Passed!
[0. 0.]

🔍 [Test 4] 全施設を使ったときの合計チェック
✅ Test 4 Passed!
[0.6 1.5]


# Text for initialize

In [244]:
# ====== 初期化関数の実行 ======
initialize_problem()

# ====== Test 1: 問題が F に1つ入っているか ======
print("🔍 [Test 1] 問題が F に初期化されているか")
if isinstance(F, list) and len(F) == 1 and isinstance(F[0], cplex.Cplex):
    print("✅ Test 1 Passed!")
else:
    print(f"❌ Test 1 Failed! F = {F}")

# ====== Test 2: x_j 変数と theta 変数が追加されているか ======
print("\n🔍 [Test 2] 変数の定義チェック")
var_names = F[0].variables.get_names()
expected_vars = [f"x{j}" for j in range(J)] + ["theta"]
if all(v in var_names for v in expected_vars):
    print("✅ Test 2 Passed!")
else:
    print(f"❌ Test 2 Failed!\nExpected: {expected_vars}\nGot: {var_names}")

# ====== Test 3: theta の目的係数が1になっているか ======
print("\n🔍 [Test 3] 目的関数の係数チェック")
theta_index = var_names.index("theta")
obj_coef = F[0].objective.get_linear()[theta_index]
if obj_coef == 1.0:
    print("✅ Test 3 Passed!")
else:
    print(f"❌ Test 3 Failed! theta の目的係数 = {obj_coef}")

# ====== Test 4: 施設数制約が正しく設定されているか ======
print("\n🔍 [Test 4] 施設数制約の右辺チェック")
constr_names = F[0].linear_constraints.get_names()
if "facility_limit" in constr_names:
    rhs = F[0].linear_constraints.get_rhs()[constr_names.index("facility_limit")]
    if rhs == p:
        print("✅ Test 4 Passed!")
    else:
        print(f"❌ Test 4 Failed! RHS = {rhs}")
else:
    print("❌ Test 4 Failed! 制約名 'facility_limit' が見つかりません")

# ====== Test 5: BestSol, theta_LB の初期化チェック ======
print("\n🔍 [Test 5] グローバル変数の初期化チェック")
if BestSol is None and theta_LB == 0:
    print("✅ Test 5 Passed!")
else:
    print(f"❌ Test 5 Failed! BestSol = {BestSol}, theta_LB = {theta_LB}")


🔍 [Test 1] 問題が F に初期化されているか
✅ Test 1 Passed!

🔍 [Test 2] 変数の定義チェック
✅ Test 2 Passed!

🔍 [Test 3] 目的関数の係数チェック
✅ Test 3 Passed!

🔍 [Test 4] 施設数制約の右辺チェック
✅ Test 4 Passed!

🔍 [Test 5] グローバル変数の初期化チェック
✅ Test 5 Passed!


# Test for generate_subsets

In [245]:
# ====== Test 1: J = 0（空集合）======
print("🔍 [Test 1] J = 0 の場合")
result = generate_subsets(0)
expected = [set()]
print("📤 出力:", result)
if result == expected:
    print("✅ Test 1 Passed!")
else:
    print(f"❌ Test 1 Failed! Expected {expected}, Got {result}")

# ====== Test 2: J = 1 ======
print("\n🔍 [Test 2] J = 1 の場合")
result = generate_subsets(1)
expected = [set(), {0}]
print("📤 出力:", result)
if all(s in result for s in expected) and len(result) == 2:
    print("✅ Test 2 Passed!")
else:
    print(f"❌ Test 2 Failed! Expected subsets: {expected}, Got: {result}")

# ====== Test 3: J = 2 ======
print("\n🔍 [Test 3] J = 2 の場合")
result = generate_subsets(2)
expected = [set(), {0}, {1}, {0,1}]
print("📤 出力:", result)
if all(s in result for s in expected) and len(result) == 4:
    print("✅ Test 3 Passed!")
else:
    print(f"❌ Test 3 Failed! Expected 4 subsets, Got: {result}")

# ====== Test 4: J = 3 ======
print("\n🔍 [Test 4] J = 3 の部分集合の個数と内容")
result = generate_subsets(3)
expected_num = 8  # 2^3 = 8
print("📤 出力:", sorted(result, key=lambda x: (len(x), sorted(x))))
if len(result) == expected_num:
    print("✅ Test 4 Passed!")
else:
    print(f"❌ Test 4 Failed! Expected {expected_num} subsets, Got: {len(result)}")

# ====== Test 5: 重複がないかチェック ======
print("\n🔍 [Test 5] 重複チェック（J = 4）")
result = generate_subsets(4)
unique_result = set(frozenset(s) for s in result)  # set of sets に変換して比較
print("📤 出力（個数）:", len(result))
print("📤 出力（部分集合）:", sorted(result, key=lambda x: (len(x), sorted(x))))
if len(unique_result) == len(result):
    print("✅ Test 5 Passed!")
else:
    print(f"❌ Test 5 Failed! 重複あり？ Got: {result}")


🔍 [Test 1] J = 0 の場合
📤 出力: [set()]
✅ Test 1 Passed!

🔍 [Test 2] J = 1 の場合
📤 出力: [set(), {0}]
✅ Test 2 Passed!

🔍 [Test 3] J = 2 の場合
📤 出力: [set(), {0}, {1}, {0, 1}]
✅ Test 3 Passed!

🔍 [Test 4] J = 3 の部分集合の個数と内容
📤 出力: [set(), {0}, {1}, {2}, {0, 1}, {0, 2}, {1, 2}, {0, 1, 2}]
✅ Test 4 Passed!

🔍 [Test 5] 重複チェック（J = 4）
📤 出力（個数）: 16
📤 出力（部分集合）: [set(), {0}, {1}, {2}, {3}, {0, 1}, {0, 2}, {0, 3}, {1, 2}, {1, 3}, {2, 3}, {0, 1, 2}, {0, 1, 3}, {0, 2, 3}, {1, 2, 3}, {0, 1, 2, 3}]
✅ Test 5 Passed!


# THE MOST SIGNIFFICANT 
# Test for compute_L_Y

## 手計算による期待値: `compute_L_Y({0}, {2})`

### 与えられた値:

- $ \text{wij\_matrix} = \begin{bmatrix} 0.1 & 0.2 & 0.3 \\ 0.4 & 0.5 & 0.6 \end{bmatrix} $
- $ U_i^L = [0.1, 0.2] $
- $ U_i^F = [0.2, 0.3] $
- $ h_i = [0.6, 0.4] $

### 条件:
- $S = \{0\},\quad Y = \{2\}$

---

### ステップ1: $ \sum_{j \in S} w_{ij} $
- 需要点 0：$ w_{00} = 0.1 $
- 需要点 1：$ w_{10} = 0.4 $

---

### ステップ2: $ \sum_{j \in S \cup Y} w_{ij} $
- $S \cup Y = \{0, 2\}$
- 需要点 0：$ w_{00} + w_{02} = 0.1 + 0.3 = 0.4$
- 需要点 1：$ w_{10} + w_{12} = 0.4 + 0.6 = 1.0$

---

### ステップ3: 各 $i$ に対して $f_i^Y$ を計算

$$
f_0 = \frac{U_0^L + \sum_{j \in S} w_{0j}}{U_0^L + U_0^F + \sum_{j \in S \cup Y} w_{0j}} 
= \frac{0.1 + 0.1}{0.1 + 0.2 + 0.4} 
= \frac{0.2}{0.7} \approx 0.2857
$$

$$
f_1 = \frac{0.2 + 0.4}{0.2 + 0.3 + 1.0} 
= \frac{0.6}{1.5} = 0.4
$$

---

### ステップ4: $L_Y(S)$ の計算

$$
L_Y(S) = h_0 f_0 + h_1 f_1 
= 0.6 \cdot 0.2857 + 0.4 \cdot 0.4 
= 0.1714 + 0.16 = 0.3314
$$

---

### ✅ 期待される結果:

$$
L_Y(S) \approx 0.3314
$$


In [246]:
# === グローバル変数の定義 ===
# 需要点: 2点 (D = 2), 施設候補: 3つ (J = 3)
wij_matrix = np.array([
    [0.1, 0.2, 0.3],  # 需要点 0 に対する施設 0,1,2 の重み
    [0.4, 0.5, 0.6]   # 需要点 1 に対する施設 0,1,2 の重み
])

Ui_L = np.array([0.1, 0.2])  # リーダーが既に開設している施設の影響
Ui_F = np.array([0.2, 0.3])  # フォロワーの既存施設の影響
h_i = np.array([0.6, 0.4])   # 各需要点の重み

In [247]:
# ====== Test 1: S = {0}, Y = {2} ======
print("🔍 [Test 1] S = {0}, Y = {2} の場合")
S = {0}
Y = {2}
try:
    result = compute_L_Y(S, Y)
    print("📤 出力 L_Y(S):", result)

    # === 手計算による期待値 ===
    # f_0 = (0.1 + 0.1) / (0.1 + 0.2 + 0.1 + 0.3) = 0.2 / 0.7 ≈ 0.2857
    # f_1 = (0.2 + 0.4) / (0.2 + 0.3 + 0.4 + 0.6) = 0.6 / 1.5 = 0.4
    # L_Y(S) = 0.6 * 0.2857 + 0.4 * 0.4 = 0.1714 + 0.16 = 0.3314
    expected = 0.3314
    if np.isclose(result, expected, atol=1e-4):
        print("✅ Test 1 Passed! 期待値:", expected)
    else:
        print(f"❌ Test 1 Failed! 期待値: {expected}, 実際の値: {result}")
except Exception as e:
    print(f"❌ Test 1 エラー発生: {e}")

# ====== Test 2: S = {}, Y = {} ======
print("\n🔍 [Test 2] S = {}, Y = {} の場合")
S = set()
Y = set()
try:
    result = compute_L_Y(S, Y)
    print("📤 出力 L_Y(S):", result)

    # f_0 = 0.1 / (0.1 + 0.2) = 0.1 / 0.3 = 0.3333
    # f_1 = 0.2 / (0.2 + 0.3) = 0.2 / 0.5 = 0.4
    expected = 0.6 * 0.3333 + 0.4 * 0.4  # ≈ 0.36
    if np.isclose(result, expected, atol=1e-4):
        print("✅ Test 2 Passed! 期待値:", expected)
    else:
        print(f"❌ Test 2 Failed! 期待値: {expected}, 実際の値: {result}")
except Exception as e:
    print(f"❌ Test 2 エラー発生: {e}")

# ====== Test 3: 分母ゼロのチェック（Ui_L, Ui_F がゼロ）======
print("\n🔍 [Test 3] Ui_L, Ui_F = 0 の場合（ゼロ割の危険性）")
Ui_L = np.array([0.0, 0.0])
Ui_F = np.array([0.0, 0.0])
S = set()
Y = set()
try:
    result = compute_L_Y(S, Y)
    print("📤 出力 L_Y(S):", result)
    print("✅ Test 3 Passed! （ゼロ割発生せず）")
except Exception as e:
    print(f"❌ Test 3 エラー発生: {e}")

# ====== Test 4: S = {0, 1}, Y = {1, 2}（共通施設あり）======
print("\n🔍 [Test 4] S = {0,1}, Y = {1,2} の場合（重複あり）")
Ui_L = np.array([0.1, 0.2])  # リセット
Ui_F = np.array([0.2, 0.3])
S = {0, 1}
Y = {1, 2}
try:
    result = compute_L_Y(S, Y)
    print("📤 出力 L_Y(S):", result)
    print("✅ Test 4 Passed!")
except Exception as e:
    print(f"❌ Test 4 エラー発生: {e}")


🔍 [Test 1] S = {0}, Y = {2} の場合
📤 出力 L_Y(S): 0.3314285714285714
✅ Test 1 Passed! 期待値: 0.3314

🔍 [Test 2] S = {}, Y = {} の場合
📤 出力 L_Y(S): 0.36
✅ Test 2 Passed! 期待値: 0.35998

🔍 [Test 3] Ui_L, Ui_F = 0 の場合（ゼロ割の危険性）
📤 出力 L_Y(S): nan
✅ Test 3 Passed! （ゼロ割発生せず）

🔍 [Test 4] S = {0,1}, Y = {1,2} の場合（重複あり）
📤 出力 L_Y(S): 0.4866666666666667
✅ Test 4 Passed!


C:\Users\adach\AppData\Local\Temp\ipykernel_32332\2638318110.py:45: RuntimeWarning: invalid value encountered in divide
  f_i_Y = numerator / denominator
